# The Project

This is a project I'm doing to parse Texas Secretary of State (SOS) voting files, develop automated cleaning scripts, identify missing fields/data and any patterns associated with that, and generally just do EDA through the whole thing.

Specific ideas for this project include:

* Want to have a way to pre-parse the SOS file to understand what data is in there / what might be missing.
* Write script to summarize raw SOS file. Want to get the following:
    1. Voters by county
        * The largest geographical region
    2. Voters by precinct
        * Counties are made up of precincts
    3. Voters by each of the [Congressional/state legislature] districts (by mapping back to precinct)
        * Use the tx-data repo for mapping to these
    4. Summarize missing fields
    
**I need to decide on the most useful way to store these data.** If the data are provided already aggregated at different levels geographically (and aren't just derived aggregations of the more resolved data), then I should push this all into a database ultimately for further analyses. However, I suspect that the data will need to be aggregated by hand, in which case a database won't be needed (would effectively be a single table if I didn't include derived data, which I wouldn't do). 

# The Data

Oof, these are not the friendliest data files in the world. First, let's establish context.

## *pir_sos_20180292*

* **Disk size =** 24.9 MB
* 36,607 records (254 county files)
* **Context:** June 30th Special Election for TX Congressional District 27 House Representative records

## *pir_sos_20180293*

* **Disk size =** 10.56 GB
* 15,531,645 records (254 county files)
* **Context:** These are individual voting records. This is where I'll focus my attention until

## *pir_sos_20180294*

* **Disk size =** 6.9 MB
* 10,101 records (13 county files)
* **Context:** 5/22/18 Democratic Primary Runoff records

## *pir_sos_20180302*

* **Disk size =** 8.7 MB
* 12,775 records (5 county files)
* **Context:** 5/22/18 Republican Primary Runoff records


It appears (from a quick view of the data) that these data files are comprised of fixed-width fields (not tab- or comma-delimited, in other words). Oh joy.

## Data Fields             
Most are self-explanatory (e.g. First Name), so here are only the ones needing some clarification:

1. County FIPS Code: numeric designation for the county in question
2. VUID: voter ID number
3. Status Code: indicates if a voter's registration is active, suspended, or cancelled. See table at bottom for mapping
4. Hispanic Surname Flag: Y = Yes
    * Assuming here that Yes means the surname given is a Hispanic 3rd or 4th name (not clear which)
5. Election Date: if data are related to a specific election, gives the date of that election here.
    * Expect this to only be non-null when data files are for a specific election, not general voter rolls
6. Election Type: e.g. "general", "primary", "runoff", etc. Table of mapped values at bottom.
    * Expect this to only be non-null when data files are for a specific election, not general voter rolls
7. Election Voting Method: e.g. Early in-person voting, election day vote, absentee ballot received, etc. Table of mapped values at bottom.
    * Expect this to only be non-null when data files are for a specific election, not general voter rolls


<br />
<br />
<center> Status Code Mappings </center>

Code | Voter Registration Status
--- | ---
V | Active
S | Suspense
C | Cancelled

<br />
<br />
<center> Election Type Mappings </center>

Code | Election Type
--- | ---
GE | General
CP | Primary
RU | Runoff
SE | Special
LO | Local
PO | Open Primary
LR | Local Runoff

<br />
<br />
<center> Election Voting Method Mappings </center>

Code | Voting Method
--- | ---
EV | Early Voting In-Person
ED | Election Day
AX | Absentee Ballot Rejected
AV | Absentee Ballot Accepted
AB | Absentee Ballot Received
PB | Provisional Ballot Accepted
PX | Provisional Ballot Rejected

In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib as plt
import seaborn as sns

In [ ]:
status_code_dict = {'V': 'Active', 'S': 'Suspense', 'C': 'Cancelled'}

election_type_dict = {'GE': 'General', 'CP': 'Primary', 'RU': 'Runoff', 'SE': 'Special',
                      'LO': 'Local', 'PO': 'Open Primary', 'LR': 'Local Runoff'}

voting_method_dict = {'EV': 'Early Voting In-Person', 
                      'ED': 'Election Day', 
                      'AX': 'Absentee Ballot Rejected', 
                      'AV': 'Absentee Ballot Accepted',
                      'AB': 'Absentee Ballot Received', 
                      'PB': 'Provisional Ballot Accepted', 
                      'PX': 'Provisional Ballot Rejected'}

In [ ]:
import voter_data_processing as vdp

voters = vdp.import_voter_data('Data/General_Voter_Rolls/', 'Data/TX_all-geography-overlap.csv')

In [ ]:
missing = vdp.calculate_missing(voters)
missing

**Since there's no obvious pattern to the Precinct Names that didn't merge, and the number of unsuccessfully merged records are 0.04% of the total record count, we're going to call this good!**

In [ ]:
voters.info(memory_usage = 'deep')

In [ ]:
voters.columns

In [ ]:
#Drop all extraneous data we don't need for analysis - e.g. Name-related data, street addresses, etc.
voters.drop(columns = ['State County Code', 'Precinct Name', 'VUID (Voter ID)', 'Last Name',
                       'First Name', 'Middle Name', 'Former Last Name', 'Suffix',
                       'Perm House Number', 'Perm Designator', 'Perm Directional Prefix', 
                       'Perm Street Name', 'Perm Street Type', 'Perm Directional Suffix', 
                       'Perm Unit Number', 'Perm Unit Type', 'Perm City', 'Mailing Address 1', 
                       'Mailing Address 2', 'Mailing City', 'Mailing State', 'Election Date',
                       'Election Type', 'Election Party', 'Election Voting Method',
                       'State House District', 'State Senate District',
                       'Hispanic Surname Flag'], inplace = True)

In [ ]:
#voter_counts = vdp.counts_by_geography(voters, folder = None)
#voter_counts['Precinct ID']

**Note:** there are approximately 15.5M records here. It's not currently clear how many of them are unique and active, but if they are all unique this represents approximately 54.9% of the TX population (out of an estimated [28,304,596 people](https://en.wikipedia.org/wiki/Texas)).

# Aggregation For Purposes of Visualization

Here's where I'll throw together different aggregated exports of the raw data, so Tableau won't have a record at the individual voter level for everyone (which would be illegal to put on the Tableau Public server, or at the very least unethical).

## First, some new fields

As a result of earlier exploration in the data, I realized there are some useful fields I can calculate from what we already have in the data, so we'll do that first before attempting to aggregate things.

In [ ]:
#pandas Timedelta type uses days as its highest unit of measure, so need to divide by a year
ONE_YEAR = pd.Timedelta(1,unit = 'Y')
ELECTION_DAY = pd.to_datetime('11/6/18')

voters['Age'] = round((ELECTION_DAY - voters['Date of Birth'])/ONE_YEAR, 0)

In [ ]:
voters['Age at Registration'] = round((voters['EDR'] - voters['Date of Birth'])/ONE_YEAR, 0)

In [ ]:
#Don't need DOB now that we have election year age
voters.drop(columns = ['Date of Birth'], inplace = True)
voters.rename({'Age': 'Election Day Age'}, inplace = True)

In [ ]:
def get_year(value):
    '''
    Meant to be used via apply() method on a pandas Series.
    Returns the year from a pandas Timestamp value
    '''
    
    if value: return value.year
    else: return value

In [ ]:
voters['Registration Year'] = voters['EDR'].apply(get_year)
voters['Registration Year'].describe()

In [ ]:
voters.drop(columns = ['EDR'], inplace = True)

In [ ]:
voters.info(memory_usage = 'deep')

In [ ]:
voters.columns

In [ ]:
#AGG STRATEGY: COUNT by Perm ZIP, age, USC District, gender, status code (decreasing number of levels per field)
group_ZIPs_Perm = voters.groupby(['Perm ZIP Code', 'Age', 'US Congressional District', 'Gender', 
                'Status Code']).count()
group_ZIPs_Perm.dropna(subset = ['_merge'], inplace = True)
group_ZIPs_Perm.to_csv('Data/Aggregated_Data/Counts_by_ZIP_Perm.csv')

In [ ]:
#Aggregation for EDR comparisons
group_EDR = voters.groupby(['Registration Year', 'Age at Registration']).count().dropna(subset = ['_merge'])
group_EDR.to_csv('Data/Aggregated_Data/Counts_by_RegistrationYear.csv')

In [ ]:
#AGG STRATEGY: COUNT by Mailing ZIP, USC District (decreasing number of levels per field)
group_ZIPs_Mailing = voters.groupby(['Mailing ZIP Code', 
                                     'US Congressional District']).count().dropna(subset = ['_merge'])
group_ZIPs_Mailing.to_csv('Data/Aggregated_Data/Counts_by_ZIP_Mailing.csv')

In [ ]:
#AGG STRATEGY: COUNT by Perm ZIP, age, USC District, gender, status code (decreasing number of levels per field)
vdp.aggregate_data(voters, ['Perm ZIP Code', 'Age', 'US Congressional District', 'Gender',
                            'Status Code'], 'Data/Aggregated_Data/Counts_by_ZIP_Perm.csv')

In [ ]:
group_ZIPs_Perm.head()

In [ ]:
group_ZIPs_Perm.index.get_level_values(0).dtype

# Voter Counts by Different Geographic Resolutions

1. By precinct
2. By state house district
3. By state senate district
4. By US Congressional district

# Analysis

OK, time to get down to business! We've done a fair amount of auditing and cleaning, memory optimization, etc. and now we have a bunch of voter data with geographic resolutions. Time to do something cool with it.

## Standard Univariate Analysis
1. Status Code
2. Gender
3. Age
4. EDR
5. Time between turning 18 and registering

In [ ]:
def percent_breakdown(x, data):
    '''
    Returns a DataFrame that provides the raw counts and percentage of the total dataset 
    of unique values from a categorical column in a pandas DataFrame
    
    Parameters
    ----------
    x: str. Name of a categorical column in a DataFrame.
    data: pandas DataFrame.
    
    Returns
    ----------
    pandas DataFrame.
    '''
    
    return pd.DataFrame({'Raw Counts': data[x].value_counts(),
                         'Percentage': data[x].value_counts()/len(data)})

### Status Code

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

t0 = time()

var = 'Status Code'
sns.barplot(x=var, y=var, orient = 'v', ci = None, data=voters, estimator=lambda x: len(x) / len(voters) * 100)
plt.ylabel("Percent")
plt.title(f"{var} Distribution")

print(f"Process completed in {round((time()-t0)/60, 2)} minutes")

In [ ]:
percent_breakdown('Status Code', voters)

**Here we're seeing roughly 87% of the voters being designated as 'V' status AKA active registration.** I'm a little surprised to see that only 13% are Suspended and that there are none in the Cancelled category. That seems odd, but OK.

### Gender

In [ ]:
t0 = time()

var = 'Gender'
sns.barplot(x=var, y=var, orient = 'v', ci = None, data=voters, estimator=lambda x: len(x) / len(voters) * 100)
plt.ylabel("Percent")
plt.title(f"{var} Distribution")

print(f"Process completed in {round((time()-t0)/60, 2)} minutes")

In [ ]:
percent_breakdown('Gender', voters)

In [ ]:
a = percent_breakdown('Gender', voters)
a.loc['M','Percentage'] + a.loc['U','Percentage'] - a.loc['F','Percentage']

**Interesting to note that the voter rolls are skewed vemale by about 6 percentage points.** Even if the Unspecified group were entirely male, females would still be 2.2 percentage points higher.

### Age

In [ ]:
voters['Date of Birth'].dtype

In [ ]:
#pandas Timedelta type uses days as its highest unit of measure, so need to divide by a year
ONE_YEAR = pd.Timedelta(1,unit = 'Y')
ELECTION_DAY = pd.to_datetime('11/6/18')

voters['Age'] = round((ELECTION_DAY - voters['Date of Birth'])/ONE_YEAR, 0)

In [ ]:
voters['Age'].describe()

In [ ]:
sns.distplot(voters['Age'].dropna())

**Well this is somewhat improbable.** Given that [the oldest recorded age in the world was just shy of 123 years](https://en.wikipedia.org/wiki/List_of_the_verified_oldest_people), we can probably ignore values above that number as being false entries.

In [ ]:
sns.distplot(voters[voters['Age'] > 123]['Age'], bins = 50)
plt.title('Distribution of Ages for Improbable Age Groups')
plt.xlim(xmin = 123)

In [ ]:
voters[voters['Age'] > 123]['Age'].count()

**Clearly, ignoring this group is not a big loss to the dataset,** especially given the small fraction of records they represent. While we're at it, let's also check in on anyone that is clearly under the age of the majority.

In [ ]:
voters[voters['Age'] < 18]['Age'].count()

In [ ]:
sns.distplot(voters['Age'].dropna(), bins = 50)
plt.title('Distribution of Ages for Probable Age Groups')
plt.xlim(18,123)

It looks like we're seeing a vaguely tri-modal distribution of ages, with peaks at roughly 30, 50, and 60.

### Effective Date of Registration (EDR)

In [ ]:
def get_year(value):
    '''
    Meant to be used via apply() method on a pandas Series.
    Returns the year from a pandas Timestamp value
    '''
    
    if value: return value.year
    else: return value

In [ ]:
voters['EDR'].describe()

**As before with DOB, we have some nonsense values.** Using again our oldest-person-in-the-world logic, I'll ignore anyone with an EDR earlier than 1913 (2018 - 123 + 18). Also, since it's currently 2018, we'll ignore anyone with an EDR after 2018.

In [ ]:
voters['Registration Year'] = voters['EDR'].apply(get_year)

In [ ]:
sns.distplot(voters['EDR - Year'].dropna(), kde = False)
plt.xlim(1913,2018)

In [ ]:
#Import US Census data from tx.gov
TX_pop = pd.read_csv('Data/TX_PopulationTrends.csv')
TX_pop.rename(columns = {'Texas Population - Count': 'Count',
                         'Texas Population - Estimate': 'Estimate',
                        'YEAR': 'Year'}, inplace = True)
TX_pop['Count'] = TX_pop['Count'].str.strip()
TX_pop['Estimate'] = TX_pop['Estimate'].str.strip()

In [ ]:
TX_pop['Estimate'] = pd.to_numeric(TX_pop['Estimate'])
TX_pop['Count'] = pd.to_numeric(TX_pop['Count'])
TX_pop.dtypes

In [ ]:
plt.subplot(2,1,1)
sns.lineplot(x = 'Year', y = 'Estimate', data = TX_pop)
plt.xlim(1913,2018)
plt.ylabel('TX Population \nEstimate')

plt.subplot(2,1,2)
sns.distplot(voters['EDR - Year'].dropna(), kde = False)
plt.xlim(1913,2018)
plt.xlabel("Year")
plt.ylabel("Count of \nVoter Registrations")

**This looks roughly similar in trend over the same time period.** At first I thought this growth in registrations was a little suspicious, but not anymore.

**It's interesting to note that more than 11% of the active voters with EDR dates in these records registered in 2016.** We can see spikes in registration every year that corresponds to a Presidential election, which isn't terribly surprising, but it is interesting to see how the registrations in each of those years appear to be growing non-linearly.

### Time Difference Between EDR and 18th Birthday

In [ ]:
(voters.loc[0]['EDR'] - voters.loc[0]['Date of Birth'])/ONE_YEAR - 18

In [ ]:
voters['18 to EDR Gap'] = (voters['EDR'] - voters['Date of Birth'])/ONE_YEAR - 18

In [ ]:
voters['18 to EDR Gap'].describe()

**Wow, really? We're seeing an average delay of 17.6 years between reaching 18 years old and registering to vote?** That's...a long time to wait. 

Although, to be fair, this is probably skewed due to people who moved into the state as an adult and thus couldn't have registered *in Texas* when they turned 18 because they didn't live there. Hmmm...this may not be as interesting of a variable as I'd hoped actually.


In [ ]:
sns.distplot(voters['18 to EDR Gap'].dropna(), bins = 50, kde = False, norm_hist = True)
plt.xlim(0, 105) #Can't be registered to vote before their 18th birthday, can't live past 123 (105 + 18)
plt.xlabel("Years Between 18th Birthday \nand Voter Registration")

**Perhaps unsurprisingly, the biggest bin is for those who registered to vote within 5 years of turning 18.** Even if you adjust this for the 2016 election, you see the same pattern (with a slightly higher perecentage in this youngest group than before).

### EDRs Pre- and Post-2016 Election

2016 General Election: 11/8/16

In [ ]:
ELECTION_DAY_2016 = pd.Timestamp(year=2016, month=11, day=9, hour=0)
voters['New Since 2016 Election'] = voters['EDR'] > ELECTION_DAY_2016

In [ ]:
percent_breakdown('New Since 2016 Election', voters)

In [ ]:
var = 'New Since 2016 Election'
sns.barplot(x=var, y=var, orient = 'v', ci = None, data=voters, estimator=lambda x: len(x) / len(voters) * 100)
plt.ylabel(f"Percent")
plt.title(f"{var} Distribution")

**Interesting, 12% of the voting population has registered since the 2016 election.** Hard to say what this means for either party, but it's an interesting piece of info.

## Counts of Active Voter Registrations
1. By Mailing address, aggregated by state/province/country
1. By precinct
1. By county
    * If I want to do this, need to merge County Names data from tx-data repo first
2. By state house district
3. By state senate district
4. By US Congressional district

### Precinct

We use the Permanent ZIP code values to geocode our voters and their Precinct IDs. Likely due to the fluid nature of ZIP codes, and a healthy dose of bad data when it comes these ZIP codes, we see a few "precincts" that are split geographically, outside the state of TX, and some precincts that have other precincts contained entirely within them. As such, these "precincts" should be viewed with mild skepticism. If something looks off about them, it likely is. Still, it is very likely that the vast majority of records are properly geolocated.

If it's worthwhile for other users, I'd recommend pulling latitude and longitude data straight from something like the GoogleV3 geocoder accessible via the [GeoPy package](https://geopy.readthedocs.io). The Google geocoder is very high accuracy, but it now costs money to use. Given that there are more than 15.5M records in these voter rolls, to use this geocoder a volume sales contract would need to be worked up, as the pay-as-you-go monthly limits only go up to 500K requests. If we assume a roughly \\$3 USD per 1000 requests rate, it would cost $46,500 to get the latitude/longitude for all of these. Other cheaper options likely exist (e.g. OpenStreetMaps API Nominatim), but the accuracy of these is lower than Google's geocoder and thus unlikely to provide a significant gain in location accuracy.

### State House

### State Senate

### US Congressional District

## *Fractions* of Active and Suspended Voter Registrations
1. By precinct
1. By county
2. By state house district
3. By state senate district
4. By US Congressional district

## Gender
1. At different geo resolutions
2. Subdivided by voter status

## Age
1. At different geo resolutions
2. Subdivided by gender
3. Subdivided by voter status

## Length of Voting Tenure

A potentially relevant measure of how many swing voters are expected is recency of their voter registration. People who are further from voting age (18 years old) who have recently registered (especially if the registration was after the last major US election, 11/8/2016) may want to "make a difference" with their vote: people upset with the outcomes from recent elections, or expecting to need to defend those outcomes, may have registered relatively recently to make their voices heard.

1. Fraction of all voters at different binned amounts of time between when they registered and when they turned 18
2. Fraction of all voters that registered since 11/8/2016
    * Breakdown by gender
    * Breakdown by age
3. Breakdown of these by geography

In [ ]:
#Push concatenated and processed voter data into a CSV file for later use
voters.to_csv('Data/Full_Voter_Data_with_Geography.csv')

In [1]:
import pandas as pd

field_types = {'Gender': 'category',
               'Perm ZIP Code': 'category',
               'Mailing ZIP Code': 'category',
               'Status Code': 'category',
               'Precinct ID': 'float32',
               'US Congressional District': 'float32',
               '_merge': 'category',
               'Age': 'float64',
               'Age at Registration': 'float64',
               'Registration Year': 'float64'}

voters = pd.read_csv('Data/Full_Voter_Data_with_Geography.csv', dtype = field_types, index_col = 0)
voters.info(memory_usage = 'deep')

/Users/emigre459/anaconda3/envs/TXVoting/lib/python3.6/site-packages/numpy/lib/arraysetops.py:522: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 15531645 entries, 0 to 15531644
Data columns (total 10 columns):
Gender                       category
Perm ZIP Code                category
Mailing ZIP Code             category
Status Code                  category
Precinct ID                  float32
US Congressional District    float32
_merge                       category
Age                          float64
Age at Registration          float64
Registration Year            float64
dtypes: category(5), float32(2), float64(3)
memory usage: 699.6 MB


In [2]:
#AGG STRATEGY: COUNT by Perm ZIP, age, USC District, gender, status code (decreasing number of levels per field)
group_ZIPs_Perm = voters.groupby(['Perm ZIP Code', 'Age', 'US Congressional District', 'Gender', 
                'Status Code'], observed = True).count()
group_ZIPs_Perm.dropna(subset = ['_merge'], inplace = True)
group_ZIPs_Perm.to_csv('Data/Aggregated_Data/Counts_by_ZIP_Perm.csv')

In [3]:
#AGG STRATEGY: COUNT by Mailing ZIP, USC District (decreasing number of levels per field)
group_ZIPs_Mailing = voters.groupby(['Mailing ZIP Code', 
                                     'US Congressional District'],
                                   observed = True).count().dropna(subset = ['_merge'])
group_ZIPs_Mailing.to_csv('Data/Aggregated_Data/Counts_by_ZIP_Mailing.csv')

In [4]:
group_ZIPs_Perm.head()

Mailing ZIP Code  \
Perm ZIP Code Age  US Congressional District Gender Status Code                     
75032         18.0 4.0                       F      V                          42   
                                             M      V                          46   
              19.0 4.0                       F      V                         171   
                                             M      S                           2   
                                                    V                         219   

                                                                 Precinct ID  \
Perm ZIP Code Age  US Congressional District Gender Status Code                
75032         18.0 4.0                       F      V                     42   
                                             M      V                     46   
              19.0 4.0                       F      V                    171   
                                             M      S                      2   
                                                    V                    219   

                                                                 _merge  \
Perm ZIP Code Age  US Congressional District Gender Status Code           
75032         18.0 4.0                       F      V                42   
                                             M      V                46   
              19.0 4.0                       F      V               171   
                                             M      S                 2   
                                                    V               219   

                                                                 Age at Registration  \
Perm ZIP Code Age  US Congressional District Gender Status Code                        
75032         18.0 4.0                       F      V                             42   
                                             M      V                             46   
              19.0 4.0                       F      V                            171   
                                             M      S                              2   
                                                    V                            219   

                                                                 Registration Year  
Perm ZIP Code Age  US Congressional District Gender Status Code                     
75032         18.0 4.0                       F      V                           42  
                                             M      V                           46  
              19.0 4.0                       F      V                          171  
                                             M      S                            2  
                                                    V                          219

In [14]:
group_ZIPs_Perm.index.get_level_values(0).categories.values

array(['00000', '75032', '75087', ..., '77987', '79001', '79092'],
      dtype=object)

In [21]:
group_ZIPs_Perm.head()

Mailing ZIP Code  \
Perm ZIP Code Age  US Congressional District Gender Status Code                     
75032         18.0 4.0                       F      V                          42   
                                             M      V                          46   
              19.0 4.0                       F      V                         171   
                                             M      S                           2   
                                                    V                         219   

                                                                 Precinct ID  \
Perm ZIP Code Age  US Congressional District Gender Status Code                
75032         18.0 4.0                       F      V                     42   
                                             M      V                     46   
              19.0 4.0                       F      V                    171   
                                             M      S                      2   
                                                    V                    219   

                                                                 _merge  \
Perm ZIP Code Age  US Congressional District Gender Status Code           
75032         18.0 4.0                       F      V                42   
                                             M      V                46   
              19.0 4.0                       F      V               171   
                                             M      S                 2   
                                                    V               219   

                                                                 Age at Registration  \
Perm ZIP Code Age  US Congressional District Gender Status Code                        
75032         18.0 4.0                       F      V                             42   
                                             M      V                             46   
              19.0 4.0                       F      V                            171   
                                             M      S                              2   
                                                    V                            219   

                                                                 Registration Year  
Perm ZIP Code Age  US Congressional District Gender Status Code                     
75032         18.0 4.0                       F      V                           42  
                                             M      V                           46  
              19.0 4.0                       F      V                          171  
                                             M      S                            2  
                                                    V                          219

In [20]:
#How many records do we have with less than 30 people having that unique combination of characteristics?
len(group_ZIPs_Perm[group_ZIPs_Perm['_merge'] < 30])

573230